In [1]:
#import部
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, sys
import glob

#警告を表示しないようにする
import warnings
warnings.filterwarnings('ignore')

##関数定義

In [2]:
#月単位グラフの前処理
#複数年のデータを、各年ごとに分割してリストに保持
def prepro_graph_monthly(df_):
    df = df_
    df_year_ = []

    for i in set(df['date'].dt.year):
        df_temp = df[df['date'].dt.year == i]
        df_year_.append(df_temp)

    return df_year_

In [3]:
#月単位グラフ描画
#目的：その年における各月のスループットの動きを確認
def graph_month(df_, outputpath):
    df = df_
    df_year_ = []
    if len(df.index) < 365:
        df_year_.append(df)
    else:
        df_year_ = prepro_graph_monthly(df)
    
    #---出力先---
    outputpath = outputpath
    outputpath_ = os.path.join(outputpath, 'monthly_')
    if not os.path.exists(outputpath_):
        os.mkdir(outputpath_) 
    #------------

    for i in range(len(df_year_)):
        df = df_year_[i]

        df_tmp_ = []
        for i in set(df['date'].dt.month):
            df_tmp = df[df['date'].dt.month==i]
            df_tmp_.append(df_tmp)

        y = []
        for num, i in enumerate(df_tmp_):
            x = list(set(df_tmp_[num]['date'].dt.day))
            y.append(i)
            plt.plot(x, y[num][y[num].columns[1]])

        title = str(list(set(df['date'].dt.year))[0])+'_'+ str(df.columns[1])+ '_'+ 'monthly_'+ outputpath 
        month_dict = {1:"Jan", 2:"Feb", 3:"Mar", 4:"Apr", 5:"May", 6:"Jun", 7:"Jul", 8:"Aug", 9:"Sep", 10:"Oct", 11:"Nov", 12:"Dec"}
        legend = list(set(df['date'].dt.month))
        legend_ = []
        for cnt in legend:
            tmp_ = month_dict[cnt] 
            legend_.append(tmp_)

        plt.title(title)
        plt.legend(legend_, loc='upper right', bbox_to_anchor=(0.5, 0.5, 0.75, 0.5))
        plt.xlabel("day")
        plt.ylabel("GTP_throughput [byte/day]")
                
        plt.savefig(outputpath_+ '/'+ title+ '.png' ,bbox_inches='tight')
        plt.clf()

In [4]:
#週単位グラフの前処理
#1年のデータを、各月ごとに分割してリストに保持
def prepro_graph_weekly(df_):
    df = df_
    
    df_month_ = []
    for i in set(df['date'].dt.month):
        df_temp = df[df['date'].dt.month == i]
        df_month_.append(df_temp)

    return df_month_

In [5]:
#weekly_の出力がすごいことになるので、整理用の関数を用意
#ファイル命名規則に乗っ取り、各年ごとに振り分け
def file_organize(targetpath_):
    targetpath = targetpath_
    os.chdir(targetpath)
    target = glob.glob('*.png')

    for i in target:
        year = i.split('_')[0]
        old = i
        new = os.path.join(year, i)
        os.renames(old,new)

In [6]:
#週単位グラフ描画
#目的：その月における各週のスループットの動きを確認
def graph_weekly(df_, outputpath):
    df = df_
    df_year_ = []
    if len(df_.index) < 365:
        df_year_.append(df)
    else:
        df_year_ = prepro_graph_monthly(df)

    #---出力先---
    outputpath = outputpath
    outputpath_ = os.path.join(outputpath, 'weekly_')
    if not os.path.exists(outputpath_):
        os.mkdir(outputpath_) 
    #------------
    
    for i in range(len(df_year_)):
        df_year = df_year_[i]
        df_month_ = prepro_graph_weekly(df_year)

        for cnt in range(len(df_month_)):
            df_ = df_month_[cnt]
            df_['days_nam'] = df_['date'].dt.weekday_name
            df_ = df_.set_index(['days_nam'])

            df_test = pd.DataFrame()
            for i in range(len(df_.index)):
                if i %7 != 0:
                    pass
                else:
                    df_test_ = df_[i:i+7]                    
                    df_test = pd.concat([df_test, df_test_], axis=1)

            cols = ['1w_datetime', '1w_data', '2w_datetime', '2w_data', '3w_datetime', '3w_data', '4w_datetime', '4w_data', '5w_datetime', '5w_data']
            df_test.columns = cols[:len(df_test.columns)]
            df_test['daynam'] = df_test.index

            days_lis = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
            df_test['daynam'] = pd.Categorical(df_test['daynam'], days_lis)

            df_plot_ = df_test.sort_values('daynam')
            df_plot = df_plot_.loc[:, df_test.columns.str.contains('data')]

            month_dict = {1:"Jan", 2:"Feb", 3:"Mar", 4:"Apr", 5:"May", 6:"Jun", 7:"Jul", 8:"Aug", 9:"Sep", 10:"Oct", 11:"Nov", 12:"Dec"}
            tmp_ = month_dict[list(set(df_['date'].dt.month))[0]]
            title =str(list(set(df_['date'].dt.year))[0])+ '_'+ str(tmp_)+ '_'+ str(df_.columns[1])+ '_'+ 'weekly_'+ outputpath

            df_plot.plot(figsize=(8,6))
            plt.title(title)
            plt.legend(loc='upper right', bbox_to_anchor=(0.5, 0.5, 0.75, 0.5))
            plt.xticks(range(len(df_plot.index)), days_lis)
            plt.xlabel("day of the week")
            plt.ylabel("GTP_throughput [byte/day]")
                        
            plt.savefig(outputpath_+ '/'+ title +'.png', bbox_inches='tight')
            plt.clf()

##データ読み込みとメイン部分

In [7]:
filelist = glob.glob('*.csv') #スクリプトと同一フォルダ内のcsvファイルを探す
print(filelist)

for i in filelist:
    output_root = i.split('.')[0]
    df = pd.read_csv(i, index_col=0)
    df['date'] = pd.to_datetime(df['date'])
    df.dtypes
    
    ##main
    df_hoge = []
    for i in df.columns[2:]:
        outputpath_ = output_root
        if not os.path.exists(outputpath_):
            os.mkdir(outputpath_)
        df_hoge = df[['date', i]]
        graph_month(df_hoge, outputpath_) #graph_month：自作関数①
        graph_weekly(df_hoge, outputpath_)#graph_weekly:自作関数②

['dammy.csv']


##出力整理

In [8]:
targetpath = input('Sort_target = ')
file_organize(targetpath)

Sort_target = /home/fuq1n0/デスクトップ/python_learning/graph_vis/dammy/weekly_


##砂場

In [ ]:
#ちょっとした処理やデータ型の確認用